In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyprind
%matplotlib inline

In [2]:
train = pd.read_csv('train.tsv', sep='\t')
train_id = pd.read_csv('train_id.tsv', sep='\t')

In [3]:
train.head()

,PERSONID,APPLYNO,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,...,FTR43,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,FTR51,CREATETIME
0,8ba94305557a4ec72435a9eb78001350,20fab21284e57a4ccd3127553920e37b,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000832,0.050000,0.0,0.0,0.000000,0.050012,0.0,0.001532,A2B186C398E0D0,2015-09-14
1,8ba94305557a4ec72435a9eb78001350,73f32b8b6e199129abb8fe0cb6d6c867,0.033333,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.002801,0.033333,0.0,0.0,0.000000,0.033333,0.0,0.005156,A24B176C1239E0D0,2015-09-14
2,8ba94305557a4ec72435a9eb78001350,324da7a0b1a5c1d3b321685d734a9dc8,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-09-14
3,8ba94305557a4ec72435a9eb78001350,341ceb4ae7b0ca1c46653f5a116d4bd2,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-10-16
4,8ba94305557a4ec72435a9eb78001350,ca260c60e62de319e38a475e2455afb8,0.050000,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,...,0.002801,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.005156,A24B176C1239E0D0,2015-10-16


In [4]:
train_id.head()

,PERSONID,LABEL
0,8ba94305557a4ec72435a9eb78001350,0
1,505d2a4db5dd353b775b5a30a731c02d,0
2,737f6cee33b47a3857b65186f5f6251e,0
3,3933908961f8e1780ff0b87cc5e4401d,0
4,c15b755463ae71dd54fa4b39d95f5170,0


In [5]:
person_count = train.groupby('PERSONID')['PERSONID'].count().to_frame(name='NUM_RECORDS').reset_index()

In [6]:
training_table = train_id.merge(person_count, on='PERSONID', how='left')

In [7]:
FTR51_ser = train['FTR51'].apply(lambda x: x.rstrip().split(','))

items_dict = dict()
for items in FTR51_ser:
    for item in items:
        if item in items_dict:
            items_dict[item] +=1
        else:
            items_dict[item] = 1

sorted_items = sorted(items_dict, key = items_dict.get)[-100:]
            
def process_list(xs):
    for idx, x in enumerate(xs):
        if x not in sorted_items:
            xs[idx] = 'OTHER'
    return xs

FTR51_ser = FTR51_ser.apply(process_list)
FTR51_str_list = [' '.join(x) for x in FTR51_ser] 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(lowercase=False, max_features=101)


X_trans = vect.fit_transform(FTR51_str_list)

X_dense=X_trans.todense()
X_df = pd.DataFrame(X_dense, columns = sorted(vect.vocabulary_, key=vect.vocabulary_.get))

train = pd.concat([train, X_df], axis=1)

In [9]:
for feature in train.columns:
    if not feature.startswith('FTR'):
        continue
    if feature == 'FTR51':
        continue
    mean_feature = train.groupby('PERSONID')[feature].mean().to_frame(name=feature+'_MEAN').reset_index()
    std_feature = train.groupby('PERSONID')[feature].std().to_frame(name=feature+'_STD').reset_index()
    max_feature = train.groupby('PERSONID')[feature].max().to_frame(name=feature+'_MAX').reset_index()
    min_feature = train.groupby('PERSONID')[feature].min().to_frame(name=feature+'_MIN').reset_index()
    training_table = training_table.merge(mean_feature, on='PERSONID', how='left').fillna(0)
    training_table = training_table.merge(std_feature, on='PERSONID', how='left').fillna(0)
    training_table = training_table.merge(max_feature, on='PERSONID', how='left').fillna(0)
    training_table = training_table.merge(min_feature, on='PERSONID', how='left').fillna(0)
    training_table[feature+'_MAXMINDIFF'] = training_table[feature+'_MAX']-training_table[feature+'_MIN']
    training_table[feature+'_MEANSTDR'] = training_table[feature+'_MEAN']/training_table[feature+'_STD']
    training_table[feature+'_MEANSTDR'] = training_table[feature+'_MEANSTDR'].replace(np.inf, 0).fillna(0)

In [10]:
unique_day = train.groupby('PERSONID')['CREATETIME'].nunique().to_frame(name='UNIQUE_DAY').reset_index()
training_table = training_table.merge(unique_day, on='PERSONID', how='left')

In [11]:
for item in sorted_items+['OTHER']:
    sum_item = train.groupby('PERSONID')[item].sum().to_frame(name=item+'_SUM').reset_index()
    training_table = training_table.merge(sum_item, on='PERSONID', how='left').fillna(0)

In [12]:
sum_ftr51 = training_table[[x+'_SUM' for x in sorted_items+['OTHER']]].sum(axis=1)
for item in sorted_items+['OTHER']:
    training_table[item+'_AVERGEDAILY'] = training_table[item+'_SUM']/training_table['UNIQUE_DAY']
    training_table[item+'_PERCENTAGE'] = training_table[item+'_SUM']/sum_ftr51

In [13]:
training_table.head()

,PERSONID,LABEL,NUM_RECORDS,FTR0_MEAN,FTR0_STD,FTR0_MAX,FTR0_MIN,FTR0_MAXMINDIFF,FTR0_MEANSTDR,FTR1_MEAN,...,A0B0C10E5D3_AVERGEDAILY,A0B0C10E5D3_PERCENTAGE,A0B0C14E0D0_AVERGEDAILY,A0B0C14E0D0_PERCENTAGE,A0B0C6E0D0_AVERGEDAILY,A0B0C6E0D0_PERCENTAGE,A0B0C0E0D0_AVERGEDAILY,A0B0C0E0D0_PERCENTAGE,OTHER_AVERGEDAILY,OTHER_PERCENTAGE
0,8ba94305557a4ec72435a9eb78001350,0,25,0.049333,0.003333,0.050000,0.033333,0.016667,14.800000,0.0,...,0.000000,0.000000,0.142857,0.027027,0.714286,0.135135,0.000000,0.000000,3.428571,0.648649
1,505d2a4db5dd353b775b5a30a731c02d,0,24,0.068889,0.102706,0.550000,0.020000,0.530000,0.670740,0.0,...,1.090909,0.073171,0.181818,0.012195,0.000000,0.000000,1.090909,0.073171,12.272727,0.823171
2,737f6cee33b47a3857b65186f5f6251e,0,75,0.069704,0.117486,0.774998,0.009092,0.765906,0.593298,0.0,...,0.393939,0.066327,0.181818,0.030612,0.303030,0.051020,0.454545,0.076531,2.818182,0.474490
3,3933908961f8e1780ff0b87cc5e4401d,0,83,0.052014,0.030872,0.325533,0.016667,0.308867,1.684858,0.0,...,0.394737,0.061475,0.078947,0.012295,0.605263,0.094262,0.394737,0.061475,3.736842,0.581967
4,c15b755463ae71dd54fa4b39d95f5170,0,28,0.049405,0.003150,0.050000,0.033333,0.016667,15.685526,0.0,...,0.000000,0.000000,0.333333,0.017021,0.000000,0.000000,0.166667,0.008511,15.250000,0.778723


In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
feature_training = training_table.drop(['PERSONID', 'LABEL'], axis=1).fillna(0)
features = feature_training.columns
X = feature_training.values
Y = training_table['LABEL'].values

In [30]:
clf = RandomForestClassifier(n_estimators=500, max_features=15)
clf.fit(X, Y)
importance_rank = np.argsort(clf.feature_importances_)[::-1]
for idx in range(50):
    print(features[importance_rank[idx]])

NUM_RECORDS
UNIQUE_DAY
OTHER_SUM
FTR34_MEAN
FTR40_MAX
FTR34_MEANSTDR
A0B0C6E0D0_SUM
FTR35_STD
FTR40_MEANSTDR
FTR35_MEAN
FTR34_STD
FTR40_STD
FTR40_MAXMINDIFF
FTR40_MEAN
FTR36_MAX
FTR36_MAXMINDIFF
FTR20_STD
FTR20_MAXMINDIFF
FTR36_MEANSTDR
FTR17_MEAN
FTR41_MEANSTDR
FTR48_MEANSTDR
FTR16_MEAN
FTR47_MEAN
FTR16_STD
FTR18_MEAN
FTR41_MEAN
FTR47_MEANSTDR
FTR0_MAXMINDIFF
FTR47_STD
FTR30_MAX
OTHER_AVERGEDAILY
FTR48_STD
FTR17_STD
FTR44_STD
FTR48_MEAN
FTR44_MEAN
FTR28_MAX
FTR30_MAXMINDIFF
FTR33_MEANSTDR
FTR20_MEAN
FTR43_MEAN
FTR9_MAX
FTR20_MEANSTDR
FTR0_MEAN
FTR32_STD
FTR43_MAX
FTR16_MEANSTDR
FTR28_MAXMINDIFF
FTR9_MEAN


In [33]:
train.groupby(['PERSONID','CREATETIME'])['APPLYNO'].nunique()

PERSONID                          CREATETIME
000e3ee6e04a8b18871e1c85ffab6e49  2015-03-05    4
                                  2015-03-06    6
                                  2015-06-03    4
                                  2015-08-02    3
                                  2015-09-13    7
                                  2015-09-18    3
                                  2015-10-11    7
                                  2015-10-17    3
                                  2015-11-01    3
                                  2015-11-13    3
                                  2015-11-14    2
                                  2015-11-15    7
                                  2015-12-11    4
                                  2015-12-20    6
                                  2016-01-07    4
                                  2016-01-22    4
                                  2016-01-30    3
                                  2016-02-05    4
                                  2016-02-19    4
000eb

In [38]:
training_table.drop(['PERSONID'], axis=1).fillna(0).corr()['LABEL'].sort_values(ascending=False)

LABEL                         1.000000
NUM_RECORDS                   0.283570
FTR40_MEANSTDR                0.256155
FTR20_MEANSTDR                0.256155
UNIQUE_DAY                    0.210370
OTHER_SUM                     0.197232
A2B107C177E0D0_SUM            0.190862
A0B0C50E0D0_SUM               0.183596
A0B0C6E0D0_SUM                0.181083
A0B0C322E0D0_SUM              0.174342
A0B0C11E0D0_SUM               0.174332
A2B143C177E0D0_SUM            0.165130
A0B0C3E0D0_SUM                0.162509
A0B0C26E0D0_SUM               0.162400
A2B79C143E0D0_SUM             0.150674
A0B0C3E0D0_AVERGEDAILY        0.148170
A0B0C36E0D0_SUM               0.148141
FTR40_MAXMINDIFF              0.126697
FTR20_MAXMINDIFF              0.126697
FTR40_MAX                     0.126685
FTR20_MAX                     0.126685
A2B259C189E0D0_SUM            0.124819
A0B3C50E0D0_SUM               0.118050
A2B70C118E0D0_SUM             0.115723
A3B14C93E0D0_SUM              0.115413
A2B107C177E0D0_AVERGEDAIL

In [39]:
training_table['FTR50_MIN']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
14970    0.0
14971    0.0
14972    0.0
14973    0.0
14974    0.0
14975    0.0
14976    0.0
14977    0.0
14978    0.0
14979    0.0
14980    0.0
14981    0.0
14982    0.0
14983    0.0
14984    0.0
14985    0.0
14986    0.0
14987    0.0
14988    0.0
14989    0.0
14990    0.0
14991    0.0
14992    0.0
14993    0.0
14994    0.0
14995    0.0
14996    0.0
14997    0.0
14998    0.0
14999    0.0
Name: FTR50_MIN, Length: 15000, dtype: float64

In [40]:
training_table.var()

LABEL                      4.870147e-02
NUM_RECORDS                4.108994e+03
FTR0_MEAN                  1.307319e-04
FTR0_STD                   1.918600e-03
FTR0_MAX                   8.818418e-02
FTR0_MIN                   1.157050e-04
FTR0_MAXMINDIFF            8.995610e-02
FTR0_MEANSTDR              3.054299e+28
FTR1_MEAN                  7.376681e-08
FTR1_STD                   7.899406e-07
FTR1_MAX                   2.783714e-05
FTR1_MIN                   0.000000e+00
FTR1_MAXMINDIFF            2.783714e-05
FTR1_MEANSTDR              3.603518e-04
FTR2_MEAN                  1.138129e-07
FTR2_STD                   6.960190e-07
FTR2_MAX                   2.017540e-05
FTR2_MIN                   2.681943e-11
FTR2_MAXMINDIFF            2.017544e-05
FTR2_MEANSTDR              6.316077e-02
FTR3_MEAN                  7.408310e-08
FTR3_STD                   7.922901e-07
FTR3_MAX                   2.778931e-05
FTR3_MIN                   0.000000e+00
FTR3_MAXMINDIFF            2.778931e-05
